# Bag of Words model for anime titles
bag of words model for titles of anime.
also should probably do it for genres too
going to be looking at the overall animelist on MAL (not user animelists) for anime titles and genres

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

### Dealing with the data
Read list of anime, list of users, and userLists, and filter them by relevant criteria. We get significant savings by discarding useless cols and by filtering userlists by whether or not user appears in our US location set.

In [2]:
# load data for anime
animelist_data = pd.read_csv('anime_list_cleaned.csv')

In [3]:
# load data for user anime lists
userlists_data = pd.read_csv('animelists_cleaned.csv') # this is big. ~2.26 GB.

In [4]:
animelist_data.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",24.0,2007.0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",24.0,2008.0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",16.0,2002.0
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)...",24.0,2012.0


In [5]:
userlists_data.head()

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:54:51,NaN
2,karthiga,74,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:43:35,NaN
3,karthiga,120,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-03 10:53:57,NaN
4,karthiga,178,26,0000-00-00,0000-00-00,7,2,0.0,0,2013-03-27 15:59:13,NaN


In [6]:
# clean userlists by removing anime that haven't been rated (they have a my_score of 0)
rated_list = userlists_data[userlists_data.my_score != 0]

In [7]:
# clean userlists further by removing unnecessary columns
rated_no_useless_cols = rated_list.drop(columns=['my_start_date', 'my_finish_date', 
                                                 'my_status', 'my_rewatching', 'my_rewatching_ep',
                                                 'my_last_updated', 'my_tags', 'my_watched_episodes'])

In [8]:
print(f'we went from {len(userlists_data)} rows down to {len(rated_no_useless_cols)}!')
print(f'the new dataset is {round(100 * len(rated_no_useless_cols)/len(userlists_data), 2)}% the size of the old one')

we went from 31284030 rows down to 19172125!
the new dataset is 61.28% the size of the old one


In [9]:
old_mem_usage = userlists_data.memory_usage(index=True).sum()
new_mem_usage = rated_no_useless_cols.memory_usage(index=True).sum()
print('with all our cleaning combined we brought the dataset down to ' + 
      str(round(100 * new_mem_usage/old_mem_usage, 2)) + '% the size of the old one!')

with all our cleaning combined we brought the dataset down to 22.29% the size of the old one!


In [10]:
# load OUR cleaned users
train = pd.read_csv('training_users_list.csv')
test = pd.read_csv('testing_users_list.csv')
US_users = pd.concat([train, test], ignore_index=True)

In [11]:
US_users.head()

,Unnamed: 0,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,join_year,age
0,11623,soultaker680,20541,4,37,0,0,2,13.873611,Male,GA,1985-04-29,NaN,2007-11-06 00:00:00,2010-08-19 11:59:00,9.63,0.0,812,2007,36.491074
1,92914,MatthewDay,1426327,8,62,10,10,230,28.088194,Male,OH,1998-04-17,NaN,2012-07-29 00:00:00,2015-05-19 23:01:00,7.96,14.0,1716,2012,23.524258
2,98181,PiercerofHeaven,2122553,8,213,8,4,20,94.853472,Male,MI,1991-11-17,NaN,2013-02-01 00:00:00,2018-05-04 22:27:44,8.04,50.0,5681,2013,29.939218
3,94973,deagosane,1486631,7,355,36,73,104,133.731250,Male,OK,1993-08-24,NaN,2012-08-30 00:00:00,2017-08-13 19:26:00,7.18,0.0,7973,2012,28.170518
4,50184,angermanaged,224902,5,68,1,1,14,39.926389,Male,IL,1990-07-05,NaN,2009-08-23 00:00:00,2012-02-19 17:26:00,6.61,0.0,2361,2009,31.308181


In [12]:
# remove unnecessary data from users dset
US_users_bare = US_users.drop(columns=['Unnamed: 0', 'user_id', 'user_watching', 'user_completed', 'user_onhold',
                                       'user_dropped', 'user_plantowatch', 'access_rank', 'last_online',
                                       'stats_rewatched', 'stats_episodes', 'birth_date', 'join_date',
                                       'stats_mean_score'])

In [13]:
print(f'memory savings of {100 - round(100 * US_users_bare.memory_usage(index=True).sum() / US_users.memory_usage(index=True).sum(), 2)}%')

memory savings of 70.0%


In [14]:
# reduce user_lists down to ONLY THOSE USERS WHO ARE IN THE US_USERS SET! 
# (where most of the savings will likely come from)
userbase = set(list(US_users_bare.username))

In [15]:
final_user_lists = rated_no_useless_cols[rated_no_useless_cols['username'].isin(userbase)]

In [16]:
print(f'we went from {len(userlists_data)} rows down to {len(final_user_lists)}!')
print(f'the new dataset is {round(100 * len(final_user_lists)/len(userlists_data), 2)}% the size of the old one')

we went from 31284030 rows down to 2825910!
the new dataset is 9.03% the size of the old one


### Bag of words model

Get the top 50 most common non-meaningless words, put them into a many-hot vector thingy, and proceed from there.

In [21]:
# vocabulary
IGNORE = {
    'of', 'the', 'and', '-', 'in', 'a', '&', 'my', 'to', 'i', 'ii', '2', '3', '4', 'is', 'on', 'no', 'x',
    'from', 'with'
}

titles = animelist_data[animelist_data['title_english'].notnull()]['title_english']
# remove quotes, colons, and trailing s's (from plurals)
begin_list = ' '.join(list(titles)).replace('"', '').replace(':', '').replace('s ', ' ').lower()

ignore_done_list = begin_list.split(' ')
ignore_done_list = [i for i in ignore_done_list if i not in IGNORE]

s = pd.Series((ignore_done_list))
top_50 = s.groupby(s).value_counts().sort_values(ascending=False).head(50)

top_50_words = [top_50.index[i][0] for i in range(50)]

### Put manyhots for top 50 words into anime dataset
Add 50 features to each anime in the anime dataset. Then, copy that dataset, dropping all columns except for `anime_id` and the BOW manyhots. This will be merged with the rest of the data.

In [22]:
relevant_cols = animelist_data[['anime_id', 'title_english']]
relevant_cols.head()

,anime_id,title_english
0,11013,Inu X Boku Secret Service
1,2104,My Bride is a Mermaid
2,5262,Shugo Chara!! Doki
3,721,Princess Tutu
4,12365,Bakuman.


In [23]:
# add a col for each of the words in the top 50
manyhots_added = pd.concat([relevant_cols, pd.DataFrame(0, index=np.arange(len(relevant_cols)), 
                                                        columns=top_50_words)], axis=1)

In [24]:
manyhots_added = manyhots_added[manyhots_added['title_english'].notnull()]

In [25]:
for i, row in manyhots_added.iterrows():
    title = row[1]
    for token in top_50_words:
        count = title.lower().split(' ').count(token)
        manyhots_added.at[i, token] = count

In [26]:
# check that the manyhots have been correctly implemented
manyhots_added.sum(axis=0).head()

anime_id                                                  44613494
title_english    Inu X Boku Secret ServiceMy Bride is a Mermaid...
movie                                                           98
girl                                                            53
special                                                         33
dtype: object

### Join (merge) datasets
Combine relevant parts of datasets to get one big messy dataset.

In [27]:
final_user_lists.dtypes

username    object
anime_id     int64
my_score     int64
dtype: object

In [28]:
US_users_bare.dtypes

username                     object
user_days_spent_watching    float64
gender                       object
location                     object
join_year                     int64
age                         float64
dtype: object

In [29]:
final_user_lists.set_index('username')

,anime_id,my_score
username,,
Damonashu,21,10
Damonashu,249,4
Damonashu,269,2
Damonashu,853,6
Damonashu,857,4
...,...,...
ThePuckishRogue,218,8
ragnogirl,8206,8
kisskaori,1486,10


In [30]:
US_users_bare.set_index('username')

,user_days_spent_watching,gender,location,join_year,age
username,,,,,
soultaker680,13.873611,Male,GA,2007,36.491074
MatthewDay,28.088194,Male,OH,2012,23.524258
PiercerofHeaven,94.853472,Male,MI,2013,29.939218
deagosane,133.731250,Male,OK,2012,28.170518
angermanaged,39.926389,Male,IL,2009,31.308181
...,...,...,...,...,...
mosaicwolf,32.678472,Female,OK,2008,45.006023
Wiseguy,103.576389,Male,CA,2008,31.105574
Ruelists,163.629167,Male,NY,2008,32.898916


In [31]:
user_lists_and_demographics = final_user_lists.merge(US_users_bare, on='username', how='left')

In [32]:
user_lists_and_demographics.head()

,username,anime_id,my_score,user_days_spent_watching,gender,location,join_year,age
0,Damonashu,21,10,82.574306,Male,MI,2008,30.234914
1,Damonashu,249,4,82.574306,Male,MI,2008,30.234914
2,Damonashu,269,2,82.574306,Male,MI,2008,30.234914
3,Damonashu,853,6,82.574306,Male,MI,2008,30.234914
4,Damonashu,857,4,82.574306,Male,MI,2008,30.234914


In [33]:
FINAL_DSET = user_lists_and_demographics.merge(manyhots_added.drop(columns=['title_english']), on='anime_id')

In [34]:
FINAL_DSET

,username,anime_id,my_score,user_days_spent_watching,gender,location,join_year,age,movie,girl,...,blade,one,super,night,cat,digimon,hero,heart,knight,episode
0,Damonashu,21,10,82.574306,Male,MI,2008,30.234914,0,0,...,0,1,0,0,0,0,0,0,0,0
1,Xinil,21,9,109.438194,Male,CA,2004,36.644398,0,0,...,0,1,0,0,0,0,0,0,0,0
2,ihasabucket,21,9,62.339583,Male,CA,2007,37.129011,0,0,...,0,1,0,0,0,0,0,0,0,0
3,WhatsUpWitches,21,9,53.766667,Male,GA,2016,25.484613,0,0,...,0,1,0,0,0,0,0,0,0,0
4,jukugo,21,2,180.754861,Male,FL,2007,33.591611,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2406539,Zoofer,12267,8,90.175347,Male,CA,2013,25.468185,0,0,...,0,0,0,0,0,0,0,0,0,0
2406540,PacificPikachu,12267,6,55.681111,Female,CA,2007,33.616252,0,0,...,0,0,0,0,0,0,0,0,0,0
2406541,Fearformychild,31201,8,173.069282,Female,AK,2015,26.897383,0,0,...,0,0,0,0,0,0,0,0,0,0
2406542,HCDman007,4339,3,19.240278,Male,TX,2015,25.284744,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# put dset into file
compression_opts = dict(method='zip', archive_name='bow_dset.csv')
FINAL_DSET.to_csv('bow_dset.zip', index=False, compression=compression_opts)